<a href="https://colab.research.google.com/github/naldo14/tesis-/blob/main/casewest_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
import h5py #librerua para crear archivos , h5py
import sys
import numpy as np
import math
from scipy import signal    #aqui estoy usando la libreria signal
import scipy.io
import os
import pandas as pd

In [2]:
#hacer el mounted de mi cuenta drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#celda para crear la lista con los paths de mi data
paths=[] #creo la lista de pahts
names=[] #creo la lista de nombres
output_file=[]
paths.append('/content/drive/MyDrive/tesis USB /case west reserve university')
names.append('casewest')
output_file.append('/content/drive/MyDrive/tesis USB /case west reserve university/data')

In [ ]:
#hago el unar del archivo
!pip install rarfile

from google.colab import drive
import rarfile

# Path to the .zip file in Google Drive
zip_file_path = paths[0]+'/'+names[0]+".rar"

with rarfile.RarFile(zip_file_path, 'r') as rf:
    rf.extractall(output_file[0])

In [ ]:
#creo que los h5py file vacios
import os
import h5py

# List of directory paths
directory_paths = [
    "/content/drive/MyDrive/tesis USB /case west reserve university/data/inner",
    "/content/drive/MyDrive/tesis USB /case west reserve university/data/outter",
    "/content/drive/MyDrive/tesis USB /case west reserve university/data/outter1",
    "/content/drive/MyDrive/tesis USB /case west reserve university/data/normal"
]

# Loop through each directory path
for directory_path in directory_paths:
    # Get the directory name (e.g., "inner," "outter," etc.)
    directory_name = os.path.basename(directory_path)

    # Create an h5py file for the directory
    h5py_file = h5py.File(f"{directory_name}.h5", "w")

    # List .mat files in the directory
    mat_files = [f for f in os.listdir(directory_path) if f.endswith(".mat")]

    # Add each .mat file to the h5py file as an empty dataset
    for mat_file in mat_files:
        # Create an empty dataset
        dataset = h5py_file.create_dataset(mat_file, shape=(0,), dtype='f4')

    # Close the h5py file
    h5py_file.close()

# Now you have separate h5py files for each directory with empty datasets

In [ ]:
#guardo los archivos . mat en los archivos h5py
# Define the h5py file name you want to inspect
h5py_file_name = "/content/drive/MyDrive/tesis USB /case west reserve university/inner.h5"
mat_files_directory='/content/drive/MyDrive/tesis USB /case west reserve university/data/inner'

# Open the h5py file
with h5py.File(h5py_file_name, 'w') as h5file:
    # Get the keys (dataset names) in the h5py file
    mat_files = [f for f in os.listdir(mat_files_directory) if f.endswith(".mat")]

    # Add each .mat file content to the h5py file
    for mat_file in mat_files:
        # Load the .mat file
        mat_data = scipy.io.loadmat(os.path.join(mat_files_directory, mat_file))

        # Create datasets for each variable in the .mat file
        for var_name, var_data in mat_data.items():
            h5file.create_dataset(f"{mat_file}/{var_name}", data=var_data)


In [ ]:

### function to storaged all the .mat file into the h5py's file
def save_mat_to_h5py(mat_files_directories, h5py_file_names):
    if len(mat_files_directories) != len(h5py_file_names):
        print("Error: The number of directories and h5py file names should match.")
        return

    for mat_files_directory, h5py_file_name in zip(mat_files_directories, h5py_file_names):
        # Open the h5py file
        with h5py.File(h5py_file_name, 'w') as h5file:
            # Get the list of .mat files in the directory
            mat_files = [f for f in os.listdir(mat_files_directory) if f.endswith(".mat")]

            # Add each .mat file content to the h5py file
            for mat_file in mat_files:
                # Load the ..mat file
                mat_data = scipy.io.loadmat(os.path.join(mat_files_directory, mat_file))

                # Create datasets for each variable in the .mat file
                for var_name, var_data in mat_data.items():
                    h5file.create_dataset(f"{mat_file}/{var_name}", data=var_data)

mat_files_directories = [
    '/content/drive/MyDrive/tesis USB /case west reserve university/data/inner',
    '/content/drive/MyDrive/tesis USB /case west reserve university/data/normal',
    '/content/drive/MyDrive/tesis USB /case west reserve university/data/outter',
    '/content/drive/MyDrive/tesis USB /case west reserve university/data/outter1'

]
h5py_file_names = [
    '/content/drive/MyDrive/tesis USB /case west reserve university/inner.h5',
    '/content/drive/MyDrive/tesis USB /case west reserve university/normal.h5',
    '/content/drive/MyDrive/tesis USB /case west reserve university/outter.h5',
    '/content/drive/MyDrive/tesis USB /case west reserve university/outter1.h5'
]
save_mat_to_h5py(mat_files_directories, h5py_file_names)

In [ ]:

file_1="/content/drive/MyDrive/tesis USB /case west reserve university/outter1.h5"
# Open the h5py file
with h5py.File(file_1, 'r') as h5file:
    # List the keys (dataset names) in the h5py file
    keys = list(h5file.keys())

    # Choose a specific .mat file (dataset) by its key
    target_mat_file = keys[1]  # Change this to the key you want to inspect

    # Access the content of the chosen .mat file
    mat_data = h5file[target_mat_file]

    # Create a dictionary to store the arrays
    arrays = {}

    for key, value in mat_data.items():
        if isinstance(value, h5py.Group):
            # If it's a group (nested data), you can access its keys
            subkeys = list(value.keys())
            for subkey in subkeys:
                # Access the actual data (arrays) and store in the dictionary
                subkey_data = value[subkey][()]
                arrays[subkey] = subkey_data
        else:
            # If it's a dataset (e.g., a variable), you can access its value
            # and store it in the dictionary using the original key
            variable_data = value[()]
            arrays[key] = variable_data

# Now, you have a dictionary 'arrays' where the keys are the array names
# and the values are the corresponding arrays from the .mat file

In [13]:

def band_maker(coeffs, fs, title):
    p = len(coeffs)- 1
    low_frequencies = [None] * len(coeffs)
    high_frequencies = [None] * len(coeffs)

    # low_frequencies[0] = 0
    # high_frequencies[0] = fs / (2 ** (p) * 2)
    for i in range(0, len(coeffs)):
         high_frequencies[i] = fs / (2 ** (p+-i + 1))
    low_frequencies[0]=0
    low_frequencies[1:len(low_frequencies)]= high_frequencies[:len(low_frequencies) - 1]



    return  high_frequencies,low_frequencies

In [14]:
def frequencies_maker(N,fs,d,D,theta):
  theta=(theta/180)*np.pi
  FTF=(fs/2)*(1-d/D*np.cos(theta))
  BPFI=(N*fs/2)*(1+d/D*np.cos(theta))
  BPFO=(N*fs/2)*(1-d/D*np.cos(theta))
  BSF=(D*fs/d)*(1-(d/D)**2*np.cos(theta))
  return(FTF,BPFI,BPFO,BSF)


In [17]:
FTF,BPFI,BPFO,BSF=frequencies_maker(9,50.17,7.90,38.5,0) #estos datos son para el daata set coreano
data = {
    'Name': ['Fundamental train frequency (FTF)', 'Ball pass frequency inner (BPFI)', 'Ball pass frequency outer (BPFO)','Ball spin frequency (BSF)'],
    'Values (HZ)': [FTF, BPFI, BPFO,BSF]
}

# Create a dataframe from the dictionary
df = pd.DataFrame(data)

# Print the dataframe
print(df)

                                Name  Values (HZ)
0  Fundamental train frequency (FTF)    19.937688
1   Ball pass frequency inner (BPFI)   272.090805
2   Ball pass frequency outer (BPFO)   179.439195
3          Ball spin frequency (BSF)   234.204744


In [8]:
# 'data' now contains a dictionary where the keys are the .mat file names
# and the values are dictionaries containing the arrays for each .mat file

def get_h5py_data(h5py_file_path):
    data = {}

    # Open the h5py file
    with h5py.File(h5py_file_path, 'r') as h5file:
        # List the keys (dataset names) in the h5py file
        keys = list(h5file.keys())

        for key in keys:
            # Access the content of the .mat file (dataset)
            mat_data = h5file[key]

            # Create a dictionary to store the arrays for this .mat file
            arrays = {}

            for subkey, value in mat_data.items():
                if isinstance(value, h5py.Group):
                    # If it's a group (nested data), you can access its keys
                    subkeys = list(value.keys())
                    for subkey in subkeys:
                        # Access the actual data (arrays) and store in the dictionary
                        subkey_data = value[subkey][()]
                        arrays[subkey] = subkey_data
                else:
                    # If it's a dataset (e.g., a variable), you can access its value
                    # and store it in the dictionary using the original key
                    variable_data = value[()]
                    arrays[subkey] = variable_data

            # Store the arrays for this .mat file in the data dictionary
            data[key] = arrays

    return data


In [9]:
## almacenamiento de la data inner
file_path = "/content/drive/MyDrive/tesis USB /case west reserve university/outter1.h5"
data_inner = get_h5py_data(file_path)

In [10]:
## almacenamiento de la data outter
file_path = "/content/drive/MyDrive/tesis USB /case west reserve university/outter.h5"
data_outter = get_h5py_data(file_path)

In [11]:
## almacenamiento de la data outter
file_path = "/content/drive/MyDrive/tesis USB /case west reserve university/outter1.h5"
data_outter_1 = get_h5py_data(file_path)

In [12]:
## almacenamiento de la data normal
file_path = "/content/drive/MyDrive/tesis USB /case west reserve university/normal.h5"
data_normal = get_h5py_data(file_path)

In [18]:
## calculo el nivel del banco de filtro:
fs=math.ceil(FTF) #frecuencia natural , se usa la del train frequencie
fe=25.6e3 # frecuencia de sampleo
n = math.ceil(math.log(fe / fs) / math.log(2)) +2
level = n